In [92]:
import mbuild as mb
import numpy as np

class CC(mb.Compound):
    def __init__(self):
        super(CC, self).__init__()

        for i in range(10):
            self.add(mb.Particle(name="C", pos=[i,0,0]))
            if i>0:
                self.add_bond((self[-2],self[-1]))
            #self.point_a = self.[0]
            #self.point_b = self.[-1]

CarbonBond = CC()
CarbonBond.visualize(show_ports=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [97]:
def end_to_end_distance(molecule):
    point_a = molecule[0].pos
    point_b = molecule[-1].pos
    return np.sqrt(sum((point_a - point_b)**2))

distance = end_to_end_distance(CarbonBond)
print(distance)

9.0
